In [ ]:
import pandas as pd
import os

import pandas as pd
import os
import numpy as np

from tqdm.notebook import tqdm

##### Date range correction
* Only dates after July 2023 will be taken

In [ ]:
# Hourly average time shiftinhg
dir = "merged"
files = [d for d in os.listdir(dir) if d.endswith('.csv')]
# subdirs = ["Badda Home"]
for file in tqdm(files, desc="Processing homes"):
    print('='*50)
    print(f"Processing {file}")
    filedir = os.path.join(dir, file)
    df = pd.read_csv(filedir, parse_dates=['UTCDateTime'])
    df["BDDateTime"] = df['UTCDateTime'] + pd.Timedelta(hours=6)
    df.to_csv(filedir, index=False)

In [ ]:
dir = "merged"
files = [d for d in os.listdir(dir) if d.endswith('.csv')]
# files = ['R4.csv']
for file in files:
    print('='*50)
    print(f"Processing {file}")
    
    df = pd.read_csv(os.path.join(dir, file), parse_dates=['BDDateTime'])
    df = df[df['BDDateTime'] >= pd.to_datetime("2023-07-01", format = "%Y-%m-%d")]
    df = df[df['BDDateTime'] <= pd.to_datetime("2025-06-30", format = "%Y-%m-%d")]
    df.to_csv(os.path.join(dir, file), index=False)

In [ ]:
## merging channel

# conditions =   {
#     'R1' : 'avg',
#     'R2' : 'b',
#     'R3' : 'avg',
#     'R4' : 'a',

#     'SU1' : 'avg',
#     'SU2' : 'avg',

#     'U1' : 'avg',
#     'U2' : 'a',
#     'U3' : 'skip',
#     'U4' : 'skip'
# }

conditions =   {
    'R1' : 'avg',
    'R2' : 'avg',
    'R3' : 'avg',
    'R4' : 'avg',

    'SU1' : 'avg',
    'SU2' : 'avg',

    'U1' : 'avg',
    'U2' : 'avg',
    'U3' : 'skip',
    'U4' : 'skip'
}

cols_to_combine = ['pm2_5_cf_1', 'pm2_5_atm', 'pm1_0_atm', 'pm10_0_atm']

## average calculation
dir = "merged"
target = 'channel_combined'
files = [d for d in os.listdir(dir) if d.endswith('csv')]
for file in tqdm(files):
    print(f'pricessing {file}')

    df = pd.read_csv(os.path.join(dir, file))
    cond = conditions[file.split('.')[0]]
    if cond == 'skip':
        print(f'Skipping {file}')
        continue
    for col in cols_to_combine:
        if cond == 'avg':
            df[f'{col}_comb'] = (df[col] + df[f'{col}_b'])/2
        elif cond == 'a':
            df[f'{col}_comb'] = df[col]
        elif cond == "b":
            df[f'{col}_comb'] = df[f'{col}_b']
        else:
            raise ValueError("cond value not recognized")

    necessary_cols = ['BDDateTime', 'current_temp_f', 'current_humidity', 'current_dewpoint_f', 'pressure', 'pm1_0_atm_comb', 'pm2_5_atm_comb', 'pm10_0_atm_comb', 'pm2_5_cf_1_comb']
    df = df[necessary_cols].copy()
    target_file_dir = os.path.join(target, file)
    df.to_csv(target_file_dir, index=False)
    print("File saved : ", target_file_dir)
    
    

##### Appling calibration

In [ ]:
## calibration
## (thersholds were applied once again here)

def calibrate_cf_1(time, pm2_5):
    if pd.isnull(time) or pd.isnull(pm2_5):
        return float('nan')
    month = time.month
    if month < 4 or month > 9:
        res =  0.63*pm2_5 - 1.1
    else:
        res =  0.77 * pm2_5 + 6.4
    
    if res >= 5 and res <= 500:
        return res
    else:
        return float('nan')
    
def calibrate_atm(time, pm2_5):
    if pd.isnull(time) or pd.isnull(pm2_5):
        return float('nan')
    month = time.month
    if month < 4 or month > 9:
        res =  0.96*pm2_5 - 2.0
    else:
        res = 1.4 * pm2_5 - 10.23

    if res >= 5 and res <= 500:
        return res
    else:
        return float('nan')
    
def limit_data(time, data):
    if pd.isnull(time) or pd.isnull(data):
        return float('nan')
    if data >= 5 and data <= 500:
        return data
    else:
        return float('nan')    


## average calculation
dir = "channel_combined"
target = 'calibrated_2min'
files = [f for f in os.listdir(dir) if f.endswith('csv')]
for file in tqdm(files):
    print(f'processing {file}')
        
    df = pd.read_csv(os.path.join(dir, file))
    df['BDDateTime'] = pd.to_datetime(df['BDDateTime'])

    df['pm2_5_cf_1_calibrated'] = df.apply(
        lambda row : calibrate_cf_1(row['BDDateTime'], row['pm2_5_cf_1_comb']), axis = 1
    )
    df['pm2_5_atm_calibrated'] = df.apply(
        lambda row : calibrate_atm(row['BDDateTime'], row['pm2_5_atm_comb']), axis = 1
    )
    df['pm1_0_atm_comb'] = df.apply(
        lambda row : limit_data(row['BDDateTime'], row['pm1_0_atm_comb']), axis = 1
    )
    df['pm10_0_atm_comb'] = df.apply(
        lambda row : limit_data(row['BDDateTime'], row['pm10_0_atm_comb']), axis = 1
    )

    df.loc[(df['pm2_5_atm_calibrated'] <= 0) | (df['pm2_5_atm_calibrated'] > 500), 'pm2_5_atm_calibrated'] = float('nan')
    df.loc[(df['pm2_5_cf_1_calibrated'] <= 0) | (df['pm2_5_cf_1_calibrated'] > 500), 'pm2_5_cf_1_calibrated'] = float('nan')

    df.drop(columns=['pm2_5_atm_comb', 'pm2_5_cf_1_comb'], inplace=True)
    df.rename(columns={
        'pm2_5_atm_calibrated' : 'pm2_5_atm',
        'pm2_5_cf_1_calibrated' : 'pm2_5_cf_1',
        'pm1_0_atm_comb' : 'pm1_0_atm',
        'pm10_0_atm_comb' : 'pm10_0_atm'
    }, inplace=True)

    df.to_csv(os.path.join(target, file), index=False)
    print("Saved", file)

#### Hourly average

In [ ]:
def hourly_averages(df, datas, time='UTCDateTime', name = None):
    results = {"time" : []}
    for col in datas:
        results[col] = []
    for col in datas:
        results[col+"count"] = []
    

    # Ensure the DataFrame has the correct time and data columns
    df = df[datas + [time]].copy()
    # df = df.dropna()  # Remove rows with NaN in time or data columns
    df[time] = pd.to_datetime(df[time])  # Ensure time column is datetime
    df = df.sort_values(by=time)  # Sort by time column

    # Generate specific hourly times for calculation (00:30, 01:30, ..., 23:30)
    start_time = df[time].min().replace(minute=30, second=0, microsecond=0)
    end_time = df[time].max().replace(minute=30, second=0, microsecond=0)
    hourly_times = pd.date_range(start=start_time, end=end_time, freq="1h")

    for target_time in tqdm(hourly_times, desc=f"Taking hourly averages : {name}"):
        # Append the results
        results["time"].append(target_time)
        # Define the 1-hour window (30 minutes before and after)
        start_window = target_time - pd.Timedelta(minutes=30)
        end_window = target_time + pd.Timedelta(minutes=30)
        # Filter data within the window
        window_data_all = df[(df[time] >= start_window) & (df[time] <= end_window)][datas]
        for data in datas:
            # Count the number of data points in the window
            window_data = window_data_all[data].dropna()
            count = len(window_data)
            # Compute the average if the number of points is >= 10
            if count >= 23:
                avg = window_data.mean()
            else:
                avg = np.nan  # Fill with NaN if insufficient data
            
            results[data].append(avg)
            results[data+'count'].append(count)

    # Convert results to a DataFrame for convenience
    results_df = pd.DataFrame(results)
    return results_df

In [ ]:
# Hourly average
dir = "calibrated_2min"
files = [f for f in os.listdir(dir) if f.endswith('.csv')]
for file in tqdm(files, desc="Processing homes"):
    print('='*50)
    print(f"Processing {file}")

    if False and os.path.exists(os.path.join(dir, subdir, f'Hourly_average_{type}.csv')):
        print("file found. skipping")
        continue
    
    df = pd.read_csv(os.path.join(dir, file), parse_dates=['BDDateTime'])
    hourly_avg = hourly_averages(df, datas=['current_temp_f', 'current_humidity', 'current_dewpoint_f', 'pressure', "pm1_0_atm", 'pm2_5_atm', "pm10_0_atm", "pm2_5_cf_1"], time='BDDateTime', name = file)
    hourly_avg.to_csv(os.path.join('hourly_averages', file), index=False)
    print(f"saved : {file}")

#### Daily Average

In [ ]:
def daily_averages(df, datas, time='UTCDateTime', name=None):
    """
    Generate daily averages from an hourly-averaged dataframe.

    Parameters
    ----------
    df : pd.DataFrame
        DataFrame containing hourly averages with a time column and one column per data in `datas`.
    datas : list[str]
        Names of the data columns to average daily (same as used in hourly_averages).
    time : str, default 'UTCDateTime'
        Name of the datetime column.
    name : str or None
        Optional label for the tqdm progress bar.

    Returns
    -------
    pd.DataFrame
        DataFrame with columns:
          - 'time' (daily timestamps at 00:00)
          - one column per data in `datas` (daily average, NaN if <= 12 hourly points available)
          - one '{data}count' column per data with the number of hourly points used that day
    """
    results = {"time": []}
    for col in datas:
        results[col] = []
    for col in datas:
        results[col + "count"] = []

    # Ensure time is datetime and sorted
    df = df.copy()
    df[time] = pd.to_datetime(df[time])
    df = df.sort_values(by=time)

    # Build daily range from min to max day (normalized to midnight)
    start_day = df[time].min().normalize()
    end_day = df[time].max().normalize()
    daily_days = pd.date_range(start=start_day, end=end_day, freq="1D")

    for day in tqdm(daily_days, desc=f"Taking daily averages : {name}"):
        # Daily window: [00:00, next day 00:00)
        start_window = day
        end_window = day + pd.Timedelta(days=1)

        results["time"].append(day)

        window_df = df[(df[time] >= start_window) & (df[time] < end_window)]

        for data in datas:
            series = window_df[data].dropna()
            count = len(series)

            # Require > 12 hourly points to compute daily average
            if count > 12:
                avg = series.mean()
            else:
                avg = np.nan

            results[data].append(avg)
            results[data + "count"].append(count)

    return pd.DataFrame(results)


In [ ]:
# Daily average
dir = "hourly_averages"
target = 'daily_averages'
files = [d for d in os.listdir(dir) if d.endswith('.csv')]
for file in tqdm(files, desc="Processing Schools"):
    print('='*50)
    print(f"Processing {file}")
    if False and os.path.exists(os.path.join(dir, subdir, f'Hourly_average_{type}.csv')):
        print("file found. skipping")
        continue
    
    df = pd.read_csv(os.path.join(dir, file))
    df['time'] = pd.to_datetime(df['time'])
    daily_avg = daily_averages(df, datas=['current_temp_f', 'current_humidity', 'current_dewpoint_f', 'pressure', "pm1_0_atm", 'pm2_5_atm', "pm10_0_atm", "pm2_5_cf_1"], time='time', name = file)
    daily_avg.to_csv(os.path.join(target, file), index=False)
    # daily_avg.info()
    print(f"saved : {file}")

#### Month avergae

In [ ]:
def monthly_averages(df, datas, time='UTCDateTime', name=None):
    """
    Generate monthly averages from a daily-averaged dataframe.

    Parameters
    ----------
    df : pd.DataFrame
        DataFrame containing daily averages with a time column and one column per data in `datas`.
    datas : list[str]
        Names of the data columns to average monthly (same as used in daily_averages).
    time : str, default 'UTCDateTime'
        Name of the datetime column.
    name : str or None
        Optional label for the tqdm progress bar.

    Returns
    -------
    pd.DataFrame
        DataFrame with columns:
          - 'time' (monthly timestamps at the first day of the month, 00:00)
          - one column per data in `datas` (monthly average, None if < 15 days available)
          - one '{data}count' column per data with the number of daily points used that month
    """

    results = {"time": []}
    for col in datas:
        results[col] = []
    for col in datas:
        results[col + "count"] = []

    # Ensure time is datetime and sorted
    df = df.copy()
    df[time] = pd.to_datetime(df[time])
    df = df.sort_values(by=time)

    # Build list of month starts (MS = Month Start)
    start_month = df[time].min().to_period('M').to_timestamp(how='start')
    end_month = df[time].max().to_period('M').to_timestamp(how='start')
    month_starts = pd.date_range(start=start_month, end=end_month, freq='MS')

    for ms in tqdm(month_starts, desc=f"Taking monthly averages : {name}"):
        # Monthly window: [month_start, next_month_start)
        next_ms = (ms + pd.offsets.MonthBegin(1))
        window_df = df[(df[time] >= ms) & (df[time] < next_ms)]

        results["time"].append(ms)

        for data in datas:
            series = window_df[data].dropna()
            count = int(series.shape[0])

            # Require at least 15 daily points to compute monthly average
            if count >= 15:
                avg = float(series.mean())
            else:
                avg = None  # keep null if insufficient data

            results[data].append(avg)
            results[data + "count"].append(count)

    return pd.DataFrame(results)


In [ ]:
# Monthly average
dir = "daily_averages"
target = 'monthly_averages'
files = [d for d in os.listdir(dir) if d.endswith('.csv')]
for file in tqdm(files, desc="Processing Schools"):
    print('='*50)
    print(f"Processing {file}")
    if False and os.path.exists(os.path.join(dir, subdir, f'Hourly_average_{type}.csv')):
        print("file found. skipping")
        continue
    
    df = pd.read_csv(os.path.join(dir, file))
    df['time'] = pd.to_datetime(df['time'])
    monthly_avg = monthly_averages(df, datas=['current_temp_f', 'current_humidity', 'current_dewpoint_f', 'pressure', "pm1_0_atm", 'pm2_5_atm', "pm10_0_atm", "pm2_5_cf_1"], time='time', name = file)
    monthly_avg.to_csv(os.path.join(target, file), index=False)
    # monthly_avg.info()
    print(f"saved : {file}")

#### Yearly Average

In [ ]:
def yearly_averages(df, datas, time='UTCDateTime', name=None):
    """
    Generate fiscal-year (Jul 1 → Jun 30) averages from a monthly-averaged dataframe.

    Parameters
    ----------
    df : pd.DataFrame
        DataFrame containing monthly averages with a time column and one column per data in `datas`.
    datas : list[str]
        Names of the data columns to average yearly (same as used in monthly_averages).
    time : str, default 'UTCDateTime'
        Name of the datetime column.
    name : str or None
        Optional label for the tqdm progress bar.

    Returns
    -------
    pd.DataFrame
        DataFrame with columns:
          - 'time' (fiscal-year start timestamps at July 1st, 00:00)
          - one column per data in `datas` (yearly average, None if < 9 monthly points in the fiscal year)
          - one '{data}count' column per data with the number of monthly points used that fiscal year
    """

    results = {"time": []}
    for col in datas:
        results[col] = []
    for col in datas:
        results[col + "count"] = []

    # Ensure time is datetime and sorted
    df = df.copy()
    df[time] = pd.to_datetime(df[time])
    df = df.sort_values(by=time)

    if df.empty:
        return pd.DataFrame(results)

    # Helper: map any date to its fiscal-year start (July 1)
    def fiscal_start(ts: pd.Timestamp) -> pd.Timestamp:
        year = ts.year if ts.month >= 7 else ts.year - 1
        return pd.Timestamp(year=year, month=7, day=1)

    # Build range of fiscal-year starts from min to max
    start_fy = fiscal_start(df[time].min())
    end_fy = fiscal_start(df[time].max())
    fy_starts = pd.date_range(start=start_fy, end=end_fy, freq='YS-JUL')  # Annual Start in July

    # Offset for next fiscal-year start
    fy_step = pd.offsets.YearBegin(1, month=7)

    for fy in tqdm(fy_starts, desc=f"Taking yearly (Jul-Jun) averages : {name}"):
        next_fy = fy + fy_step
        window_df = df[(df[time] >= fy) & (df[time] < next_fy)]

        results["time"].append(fy)

        for data in datas:
            series = window_df[data].dropna()
            count = int(series.shape[0])

            # Require at least 9 monthly points (~75%) within the fiscal year
            if count >= 6:
                avg = float(series.mean())
            else:
                avg = None  # insufficient data for fiscal year average

            results[data].append(avg)
            results[data + "count"].append(count)

    return pd.DataFrame(results)


In [ ]:
# Monthly average
dir = "monthly_averages"
target = 'yearly_averages'
files = [d for d in os.listdir(dir) if d.endswith('.csv')]
for file in tqdm(files, desc="Processing Schools"):
    print('='*50)
    print(f"Processing {file}")
    if False and os.path.exists(os.path.join(dir, subdir, f'Hourly_average_{type}.csv')):
        print("file found. skipping")
        continue
    
    df = pd.read_csv(os.path.join(dir, file))
    df['time'] = pd.to_datetime(df['time'])
    yearly_average = yearly_averages(df, datas=['current_temp_f', 'current_humidity', 'current_dewpoint_f', 'pressure', "pm1_0_atm", 'pm2_5_atm', "pm10_0_atm", "pm2_5_cf_1"], time='time', name = file)
    yearly_average.to_csv(os.path.join(target, file), index=False)
    # yearly_average.info()
    print(f"saved : {file}")

###